In [7]:
# Import required modules
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Create variables for scraped data
OFF_TEAM = []
DEF_TEAM = []
FPTS = []
WEEK = []
pos_dict = {'QB':1,'WR':2,'RB':3,'TE':4,'K':7,'DEF':8}
pos = pos_dict['QB']
week_num = 2

for i in range(week_num):

    url = "https://fantasy.nfl.com/research/pointsagainst?position=" + str(pos) + "&statCategory=pointsAgainst&statSeason=2021&statType=weekPointsAgainst&statWeek=" + str(i+1)
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'lxml')
    time.sleep(1)

    # Create an object of the first object that is class=data compact
    table = soup.find('tbody')

    # Find all the <tr> tag pairs, skip the first one, then for each.
    for row in table.find_all('tr'):
        
        # Create a variable of all the <td> tag pairs in each <tr> tag pair,
        #col = row.find('td').text

        DEF_TEAM.append(row.find_all('td')[0].text)
        OFF_TEAM.append(row.find_all('td')[1].text)
        FPTS.append(row.find_all('td')[2].text)
        WEEK.append(i+1)

    # Create a variable of the value of the columns
col = {'OFF_TEAM': OFF_TEAM,
       'DEF_TEAM': DEF_TEAM,
       'FPTS': FPTS,
       'WEEK': WEEK,
       }

    # Create a dataframe from the columns variable
df = pd.DataFrame(col)
df['POSITION'] = df['DEF_TEAM'].apply(lambda x: x.split('vs ')[1])
df['DEF_TEAM'] = df['DEF_TEAM'].apply(lambda x: x.split('vs ')[0]).str.replace(" Defense", "")
df['OFF_TEAM'] = df['OFF_TEAM'].apply(lambda x: x.replace("@", ""))
#df['DEF_TEAM'] = df['DEF_TEAM'].apply(lambda x: x.replace("WAS", "WSH"))
#df['DEF_TEAM'] = df['DEF_TEAM'].apply(lambda x: x.replace("LA", "LAR"))
df['OFF_TEAM'] = df['OFF_TEAM'].apply(lambda x: 'WSH' if x=='WAS' else x)
df['OFF_TEAM'] = df['OFF_TEAM'].apply(lambda x: 'LAR' if x=='LA' else x)
#df = df[df.TEAM.str.contains("vs ")]
#df["OFF_TEAM"], df["POSITION"], df["VS"] , df["DEF_TEAM"], df["JUNK"] = zip(*df["TEAM"].str.split().tolist())
df = df[["WEEK", "POSITION", "OFF_TEAM", "DEF_TEAM" , "FPTS"]]
df['FPTS'] = df['FPTS'].astype(float).astype(int)
df.to_csv("FF_Data", index=False)
df

,WEEK,POSITION,OFF_TEAM,DEF_TEAM,FPTS
0,1,QB,ARI,Tennessee Titans,33
1,1,QB,KC,Cleveland Browns,33
2,1,QB,NO,Green Bay Packers,29
3,1,QB,DET,San Francisco 49ers,28
4,1,QB,PHI,Atlanta Falcons,28
...,...,...,...,...,...
59,2,QB,DAL,Los Angeles Chargers,7
60,2,QB,NE,New York Jets,7
61,2,QB,JAX,Denver Broncos,6
62,2,QB,MIA,Buffalo Bills,6


In [9]:
import pandas as pd
df = pd.read_csv("FF_Data")
df2 = pd.read_csv("Team Index.csv")
schedule = pd.read_csv("Team Schedule.csv")
team_index = df2[['Abb','Team']]

melted_schedule = pd.melt(schedule, var_name = 'Week', value_name='Team_ID')
melted_schedule = melted_schedule[melted_schedule['Week'] != 'TEAM']
home_teams = pd.merge(melted_schedule, team_index, left_on=['Team_ID'], right_on=['Abb'])
home_teams['Week'] = home_teams['Week'].astype(int)

data = pd.merge(df, df2, how='inner', left_on = 'OFF_TEAM', right_on = 'Abb')
data = data.rename(columns={'Index': 'OFF_Index'})
data = pd.merge(data, df2, how='inner', left_on = 'DEF_TEAM', right_on = 'FullName')
data = data.rename(columns={'Index': 'DEF_Index'})
data = pd.merge(data, data[['OFF_Index','FPTS', 'WEEK']], how='inner', left_on = ['DEF_Index', 'WEEK'], right_on = ['OFF_Index', 'WEEK'])
data = pd.merge(data, home_teams, how='inner', left_on=['Abb_x','WEEK'], right_on = ['Abb','Week'])
data = data[['DEF_Index', 'OFF_Index_x', 'FPTS_y', 'FPTS_x']]
data.columns = ['HOME', 'AWAY', 'HOME_FPTS', 'AWAY_FPTS']
games = data.values.tolist()
data

,HOME,AWAY,HOME_FPTS,AWAY_FPTS
0,30,7,14,33
1,5,29,17,22
2,25,9,32,24
3,20,19,26,18
4,21,26,18,9
5,21,18,19,18
6,14,13,7,28
7,6,11,27,27
8,6,14,29,16
9,8,11,14,7


In [10]:
import numpy as np
def buildGamesMatrix2(games, num_teams):
    l = len(games)
    M = np.zeros([l*2, num_teams*2+1])
    for i in range(l):
        g = games[i]
        M[i, g[0]] += 1
        M[i, g[1]+num_teams] += -1
        M[i, num_teams*2] += 1
        M[l+i, g[1]] += 1
        M[l+i, g[0]+num_teams] += -1
    return M

M2 = buildGamesMatrix2(games,32)
print(M2)

[[0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [11]:
def buildOutcomes2(games, n):
    l = len(games)
    E = np.zeros([l*2])
    for i in range(l):
        g = games[i]
        E[i] += g[2]
        E[l+i] += g[3]
    return E

E2 = buildOutcomes2(games, 32)
print(E2)

[14. 17. 32. 26. 18. 19.  7. 27. 29. 14. 13. 18. 16. 12.  6. 18. 33. 20.
 11. 21. 28. 21. 23.  2. 29. 23. 24. 15. 27. 33.  6. 17. 33. 22. 24. 18.
  9. 18. 28. 27. 16.  7. 24. 27. 17. 10. 21. 22. 25. 29. 13. 20. 19. 19.
 19.  7.  2. 14. 12. 16. 18. 12. 16. 12.]


In [13]:
from sklearn import linear_model
clf = linear_model.Ridge(fit_intercept=False, alpha=0.1)
clf.fit(M2,E2)

R = clf.coef_

results_df = pd.DataFrame()
schedule2 = pd.read_csv("Team Schedule.csv")
WEEK = 3

for i in range(32):
    temp = pd.DataFrame({'Team': i, 'For': R[i], 'Against': R[i+32]}, index=[0])
    results_df = pd.concat([results_df, temp])

results_df = pd.merge(results_df, df2, how='inner', left_on = 'Team', right_on = 'Index')
results_df = pd.merge(results_df, schedule2, how='inner', left_on = 'Abb', right_on = 'TEAM')
results_df.rename(columns={str(WEEK): 'OPP'}, inplace=True)
results_df['OPP'] = results_df['OPP'].replace({'@':''}, regex=True)
results_df = results_df[['TEAM', 'Index', 'For', 'Against', 'OPP']]
results_df = pd.merge(results_df, results_df, left_on = 'TEAM', right_on = 'OPP')
results_df['Projected'] = results_df['For_x'] - results_df['Against_y']
results_df['Position'] = pos
results_df = results_df[['Position', 'TEAM_x', 'For_x', 'OPP_x', 'Against_y', 'Projected']]
results_df.sort_values(by = 'Projected', ascending=[0])

#save and/or append results to csv
#results_df.to_csv('FF_Results.csv', mode='a', header=False, index = False)
#results_df.to_csv('FF_Results.csv', header=True, index = False)


,Position,TEAM_x,For_x,OPP_x,Against_y,Projected
16,1,NYG,16.294457,ATL,-18.360210,34.654667
25,1,BAL,18.616455,DET,-13.221966,31.838422
7,1,ARI,20.222028,JAX,-10.233458,30.455486
21,1,CAR,14.146080,HOU,-12.671411,26.817491
24,1,LAR,7.493563,TB,-19.290723,26.784286
31,1,MIN,16.780168,SEA,-9.600200,26.380368
23,1,LV,15.372676,MIA,-7.885851,23.258527
0,1,SF,7.419275,GB,-15.323843,22.743118
27,1,SEA,11.937349,MIN,-10.301739,22.239088
20,1,GB,6.226795,SF,-13.979623,20.206418
